In [1]:
WEB_SITE_matrix = './chetor.com/view_matrix/lil_matrix.npz'
WEB_SITE_CONTENT_CSV = './pages_chetor.csv'
WEB_SITE_itemindexer = './chetor.com/view_matrix/item_indexer.indexer'
WEB_SITE_userindexer = './chetor.com/view_matrix/user_indexer.indexer'
SAVING1 = './chetor.com/alisResult/ALS/ali_item_indexer_factorized.indexer'
SAVING2 = './chetor.com/alisResult/ALS/ali_user_indexer_factorized.indexer'
SAVING3 = './chetor.com/alisResult/ALS/ali_items_vectors.npy'
SAVING4 = './chetor.com/alisResult/ALS/ali_users_vectors.npy'
WEB_SITE_NAME = 'chetor.com'
RESULTS_PATH = '/home/yektanet/Pictures/annoy_opt/'
PARTITION = 0.5 # max 1

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from tqdm import tqdm

import timeit
import time
import math

# ./indexer
from indexer import AppendIndexer
import ALS

# Annoy
from annoy import AnnoyIndex

#SKLearn 
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import average_precision_score
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

# scipy
from scipy.spatial import distance
from scipy.sparse import lil_matrix

In [3]:
WEBSITE_content = pd.read_csv(WEB_SITE_CONTENT_CSV)
print(WEBSITE_content.shape)
WEBSITE_content.head(50)

(9640, 15)


,id,created,content_vector,title,cleaned_url,website_id,included,is_amp,has_image_meta,has_title_meta,page_id,subdomain_id,title_vector,image_id,cats
0,2177439,2020-08-09 04:51:12.676435+04:30,"{0.52510529756546,-0.51316374540329,-0.3844181...",وردپرس › خطا,chetor.com/,1,f,f,f,f,148948,336,"{-0.90155690908432,-1.60752034187317,0.1689879...",1015133.0,{}
1,973115,2020-08-09 03:50:47.939727+04:30,"{-0.111537165939808,0.314831465482712,-0.60968...",تبلیغات در وب سایت چطور | چطور,chetor.com/%D8%AA%D8%A8%D9%84%DB%8C%D8%BA%D8%A...,1,t,f,f,t,27016495,336,"{-1.89593696594238,0.659652948379517,-0.852037...",NaN,{}
2,975692,2020-08-09 03:50:49.22613+04:30,"{0.628654301166534,-0.548453748226166,-0.25651...",همکاری با ما | چطور,chetor.com/%D9%87%D9%85%DA%A9%D8%A7%D8%B1%DB%8...,1,t,f,f,t,27004594,336,"{-2.04737234115601,2.28955912590027,-2.1406197...",333341.0,{}
3,2144818,2020-08-09 04:49:41.361098+04:30,"{0.350972026586533,-0.0532753616571426,-0.2103...",چرا باید توسعه فردی را جدی بگیرید؟- چطور,chetor.com/100-%DA%86%D8%B1%D8%A7-%D8%A8%D8%A7...,1,t,f,f,f,5274753,336,"{2.27225732803345,0.7027268409729,0.4857064485...",599277.0,{}
4,2157999,2020-08-09 04:50:09.423651+04:30,"{1.79109728336334,-1.05194425582886,-1.3205879...",دلایل سرد بودن دستان چیست؟ | چطور,chetor.com/100026-%D8%B3%D8%B1%D8%AF-%D8%A8%D9...,1,t,f,f,f,2416997,336,"{2.48906230926514,-1.19906878471375,-1.4995402...",1485027.0,{}
5,4465070,2020-10-06 11:10:31.622853+03:30,"{0.683604598045349,-0.237597540020943,0.257590...",افسوس های من | چطور,chetor.com/100041-%D8%A7%D9%81%D8%B3%D9%88%D8%...,1,t,f,t,t,38243000,336,NaN,1012527.0,{}
6,2142257,2020-08-09 04:49:40.250501+04:30,"{0.63472980260849,0.145868331193924,0.12041924...",خانه پدربزرگ | چطور,chetor.com/100044-%D8%AE%D8%A7%D9%86%D9%87-%D9...,1,t,f,f,f,6160851,336,"{1.89609825611115,1.75975489616394,2.689968109...",1012527.0,{}
7,2123739,2020-08-09 04:48:50.486508+04:30,"{0.63071072101593,-0.431413412094116,-0.302038...",درددل های زیادی در دل اطرافیانم مانده | چطور,chetor.com/100049-%D8%AF%D8%B1%D8%AF%D8%AF%D9%...,1,t,f,f,f,11269955,336,"{1.16336917877197,0.0402661710977554,0.3318648...",NaN,{}
8,2161889,2020-08-09 04:50:45.622368+04:30,"{2.3618381023407,-0.220653653144836,-1.4399104...",درمان سردرد و تسکین فوری آن با ۱۴ روش موثر و ی...,chetor.com/100085-%D8%AF%D8%B1%D9%85%D8%A7%D9%...,1,t,f,f,f,2262178,336,"{2.5592200756073,0.69496887922287,-2.160656929...",1013280.0,{}
9,2161493,2020-08-09 04:50:45.464757+04:30,"{1.23862218856812,-0.99531877040863,-0.5665395...",آیا موز چاق کننده است؟ | چطور,chetor.com/100187-%D8%A2%DB%8C%D8%A7-%D9%85%D9...,1,t,f,f,f,2273389,336,"{1.81400036811829,-1.17559599876404,-1.4106029...",1013170.0,{}


In [4]:
WEBSITE_content = WEBSITE_content[WEBSITE_content.included == 't']
WEBSITE_content.shape

(8723, 15)

In [5]:
class ViewMatrix:
    def __init__(self, path):
        self.path = path
        self.original = True
        self.item_indexer = AppendIndexer.load(WEB_SITE_itemindexer)
        self.user_indexer = AppendIndexer.load(WEB_SITE_userindexer)
        
    def load_matrix(path):
        
        matrix = ViewMatrix(path)

        try:
            matrix.view_matrix = ViewMatrix \
                .load_sparse_lil(path)
        except:
            print('Error: loading', path)

        return matrix

    def load_sparse_lil(filename):
        loader = np.load(filename, allow_pickle=True)
        result = lil_matrix(tuple(loader["shape"]), dtype=str(loader["dtype"]))
        result.data = loader["data"]
        result.rows = loader["rows"]
        return result
    
    def make_dense(self, user_min_view, item_min_view):
        self.original = False
        while True:
            removed_rows_cnt = self.trim_users_with_few_views(user_min_view)
            removed_columns_cnt = self.trim_columns_with_few_views(item_min_view)
            if not removed_columns_cnt and not removed_rows_cnt:
                break

    def trim_users_with_few_views(self, user_min_view):
        removing_row_indices = list(np.where(self.view_matrix.getnnz(1) < user_min_view)[0])
        print('Number of users which should be deleted:', len(removing_row_indices))
        self.trim_user_indices(to_remove_indices=removing_row_indices)
        return len(removing_row_indices)

    def trim_columns_with_few_views(self, column_min_view):
        removing_column_indices = list(np.where(self.view_matrix.getnnz(0) < column_min_view)[0])
        print('Number products which should be deleted:', len(removing_column_indices))
        self.trim_column_indices(to_remove_indices=removing_column_indices)
        return len(removing_column_indices)
    
    def trim_user_indices(self, to_remove_indices):
        self.user_indexer.remove_indexes(to_remove_indices)
        self.view_matrix = ViewMatrix.delete_row_lil(self.view_matrix, to_remove_indices)
    
    def trim_column_indices(self, to_remove_indices):
        self.item_indexer.remove_indexes(to_remove_indices)
        self.view_matrix = ViewMatrix.delete_column_lil(self.view_matrix, to_remove_indices)
    
    def delete_column_lil(mat: lil_matrix, *i) -> lil_matrix:
        mat = mat.transpose()
        mat = ViewMatrix.delete_row_lil(mat, *i)
        return mat.transpose()
    
    def delete_row_lil(mat: lil_matrix, *i) -> lil_matrix:
        if not isinstance(mat, lil_matrix):
            raise ValueError("works only for LIL format -- use .tolil() first")
        mat = mat.copy()
        mat.rows = np.delete(mat.rows, i)
        mat.data = np.delete(mat.data, i)
        mat._shape = (mat.rows.shape[0], mat._shape[1])
        return mat
    def to_csr(self):
        train_data = self.view_matrix.astype(np.float64)
        train_data = train_data.tocoo()
        train_data.data = np.log10(train_data.data) + 1
        train_data = train_data.tocsr()
        return train_data

In [6]:
def CFTrain(path: str, _alpha = 15, factors = 20, _test_size = PARTITION):
    now = time.time()
    matrix = ViewMatrix.load_matrix(path)
    print('View matrix loaded in', time.time() - now, 'seconds.')

    now = time.time()
    sparce_matrix = matrix.to_csr()
    matrix.make_dense(user_min_view = 3, 
                      item_min_view = 3)
    print('matrix has been made dense in', time.time() - now, 'seconds.')

    als_model = ALS.Als(num_factors = 65,
                        iterations = 15,
                        num_threads = 15,
                        alpha = 200)

    implicit_matrix = matrix.to_csr()
    
    now = time.time()
    als_model.fit(implicit_matrix)
    alsTime = time.time() - now
    print('ALS model is fitted in', time.time() - now, 'seconds.')
    
    print('Saving Data ...')
    matrix.item_indexer.dump(SAVING1)
    matrix.user_indexer.dump(SAVING2)
    np.save(SAVING3, als_model.item_vectors)
    np.save(SAVING4, als_model.user_vectors)
    
    return sparce_matrix, implicit_matrix, als_model.item_vectors, als_model.user_vectors, alsTime

In [7]:
sparce_matrix, implicit_matrix, item_vectors, user_vectors, alsTime = \
CFTrain(WEB_SITE_matrix, factors = 20)

View matrix loaded in 9.157627582550049 seconds.
Number of users which should be deleted: 2038106
Number products which should be deleted: 2785
Number of users which should be deleted: 685
Number products which should be deleted: 53
Number of users which should be deleted: 51
Number products which should be deleted: 10
Number of users which should be deleted: 10
Number products which should be deleted: 0
Number of users which should be deleted: 0
Number products which should be deleted: 0
matrix has been made dense in 12.607528686523438 seconds.
ALS model is fitted in 111.01111245155334 seconds.
Saving Data ...


In [8]:
item_indexer = AppendIndexer.load(SAVING1)
user_indexer = AppendIndexer.load(SAVING2)

In [9]:
ids = WEBSITE_content['id'].tolist()

In [10]:
ratings = user_vectors.dot(item_vectors.T)

In [11]:
print('sparce matrix:\t', sparce_matrix.shape)
print('implicit matrix:', implicit_matrix.shape)
print('item vectors:\t', item_vectors.shape)
print('users vectors:\t', user_vectors.shape)

sparce matrix:	 (2097150, 8190)
implicit matrix: (58298, 5342)
item vectors:	 (5342, 65)
users vectors:	 (58298, 65)


In [12]:
print('user tokens:\t', np.array(list(user_indexer.get_items())).shape)
print('item ids:\t', np.array(list(item_indexer.get_items())).shape)
print(ratings.shape)

user tokens:	 (58298,)
item ids:	 (5342,)
(58298, 5342)


In [13]:
row, col = implicit_matrix.nonzero()
for i, j in zip(row, col):
    ratings[i, j] = 0
users_recoms = []
for user_ratings in tqdm(ratings):
    recoms = np.argsort(user_ratings)[-10:][::-1]
    user_recoms = []
    for r in recoms:
        rec = item_indexer.reverse_get(r)
        if int(rec) in ids:
#             user_recoms.append(r)
            user_recoms.append(rec)
    users_recoms.append(user_recoms)

100%|██████████| 58298/58298 [01:04<00:00, 910.39it/s] 


In [14]:
users = list(user_indexer.get_items())
print(len(users_recoms), len(list(user_indexer.get_items())))

58298 58298


In [15]:
def check(user_num = 100):
    print('---user:', users[user_num], '---')
    print('what user saw:')
    for i in col[row == user_num]:
        print(WEBSITE_content[WEBSITE_content['id'] == int(item_indexer.reverse_get(i))].title.values)
    print('what we recommend:')
    for i in users_recoms[user_num]:
        print(WEBSITE_content[WEBSITE_content['id'] == int(i)].title.values)

In [16]:
for i in range(80,50000, 220):
    check(i)

---user: 1ba07595-6d36-41e7-c149-6176e26909b6 ---
what user saw:
['بهترین فیلم های ترسناک دنیا که دلهره و ترس را به جان\u200cتان می\u200cاندازند | چطور']
['درمان کورک با ۷ روش خانگی مؤثر | چطور']
['نکاتی که رعایت آنها عوارض شب بیداری را کاهش می\u200cدهد | چطور']
what we recommend:
['۴۴ فیلم دیدنی از بهترین فیلم های جنگی تاریخ سینما | چطور']
['بهترین فیلم های تاریخی جهان که حقیقت را به شما نمایش می\u200cدهند | چطور']
['۲۵ فیلم برتر قرن ۲۱ تاكنون كه حتما باید ببینید | چطور']
['انواع قرص خواب؛ ۸ نوع قرص خواب آور و نحوهٔ مصرف آن\u200cها | چطور']
['بهترین فیلم های رازآلود جهان که ذهن شما را ساعت\u200cها درگیر خود می\u200cکنند | چطور']
['بهترین فیلم های عاشقانه جهان که شما را در فضایی رمانتیک غرق می\u200cکنند | چطور']
['بهترین فیلم های تاریخ سینما که حداقل یک بار باید ببینید | چطور']
['بهترین فیلم های اکشن؛ ۳۱ فیلم برتر برای دوستداران هیجان | چطور']
['بهترین فیلم های علمی تخیلی از گذشته تا کنون | چطور']
['بهترین فیلم های 2018 که شما را پای تلویزیون میخکوب می\u200cکنند | چطور']
---user: t2r6c

['انواع بیماری های کبد، علائم، روش\u200cهای تشخیص و درمان | چطور']
['علائم سیروز کبد، شیوه\u200cهای تشخیص و درمان آن | چطور']
['عوارض کمبود ویتامین D و راههای درمان آن | چطور']
['علائم، راه\u200cهای درمان و پیشگیری از کبد چرب | چطور']
['بیماری ها | چطور']
['کیسه صفرا؛ ۱۰ مشکل شایع کیسه صفرا، علائم و درمان آنها | چطور']
['منابع ویتامین D؛ غنی\u200cترین خوراکی\u200cهایی که ویتامین D را تامین می\u200cکنند | چطور']
['همه چیز درباره ویتامین E | چطور']
['رژیم و تغذیه | چطور']
---user: 51359d79-668f-45a3-c56f-eef69a014dec ---
what user saw:
['۱۹ نمونه سوال زیرکانه که در مصاحبه\u200c شغلی از شما می\u200cپرسند | چطور']
['آموزش مدیتیشن برای تازه\u200cکارها با ۲۰ نکته عملی | چطور']
['نمونه سوالات روانشناسی در مصاحبه استخدامی؛ خودتان را برای این سوالات آماده کنید | چطور']
what we recommend:
['حرف\u200cهایی که هرگز نباید در مصاحبه شغلی بگویید | چطور']
['مصاحبه و رزومه | چطور']
['10 سؤال سخت مصاحبه\u200cهای شغلی و جواب\u200cهای دندان\u200cشکن به آنها | چطور']
['نمونه سوالات مصاحبه\u200cهای شغلی و پا

['۱۸ فیلم تأثیرگذاری که می\u200cتواند زندگی هر زنی را تغییر دهند | چطور']
---user: 86726384-0cb3-4711-c7ed-2f1773e8502d ---
what user saw:
['۶۳ روش برای بالا بردن اعتماد به نفس | چطور']
['چگونه با شخصیت باشیم؟ | چطور']
['چگونه مهربان باشیم؟ | چطور']
what we recommend:
['تقویت اعتماد به نفس با ۶ راهکار تضمینی | چطور']
['چگونه باکلاس باشیم؛ ۲۶ نکته برای شیک برخورد کردن | چطور']
['چگونه محبوب باشیم؛ ۲۱ توصیه\u200c برای جا باز کردن در دل دیگران | چطور']
['۱۰ نکته شگفت\u200cانگیزی که بر جذابیت رفتاری شما \u200cمی\u200cافزاید | چطور']
['چگونه خاص باشیم؛ نکاتی که شما را برجسته و متفاوت نشان می\u200cدهد | چطور']
['۶ اخلاق سمّی که آدم\u200cها را از شما می\u200cراند | چطور']
['چگونه شخصیت قوی داشته باشیم؟ | چطور']
['چگونه بهترین باشیم؛ ۱۷ کاری که شما را از معمولی بودن نجات می\u200cدهد | چطور']
['تغییر شخصیت با ۱۴ توصیه کمتر شنیده شده | چطور']
['علائم کمبود اعتماد به نفس چیست؟ | چطور']
---user: a046a66d-d3935-5888d-d14ce-eda9932f455bf ---
what user saw:
['منابع ویتامین D؛ غنی\u200cترین خوراکی\u20

['چرا قوانین وجود دارند | چطور']
what we recommend:
['امتحان و کنکور | چطور']
['حفظ سریع مطالب در ۹ گام ساده اما موثر | چطور']
['افزایش سرعت یادگیری با ۵ راهکار واقعا نتیجه\u200cبخش | چطور']
['چند نکته برای درس خواندن صحیح؛ چطور همیشه آماده باشیم | چطور']
['چگونه درس بخوانیم؟ | چطور']
['چگونه درس بخوانیم که خسته نشویم | چطور']
['آشنایی با موثرترین روش های مطالعه و یادداشت برداری | چطور']
['کاربردی\u200cترین روش\u200cهای تقویت اراده برای درس خواندن | چطور']
['راهکارهای افزایش انگیزه برای درس خواندن | چطور']
['۱۶ روش درمان حالت تهوع و علت ایجاد آن | چطور']
---user: 020f06ba-8bab-40a2-c0d8-b21f2d27d36f ---
what user saw:
['۴ تکنیک برای تقویت هوش هیجانی | چطور']
['تاثیر رنگ در تبلیغات و بازاریابی چقدر است؟ | چطور']
['استراتژی تبلیغات چیست؛ معرفی ۶ استراتژی مهم تبلیغاتی در تجارت\u200c\u200cهای کوچک | چطور']
what we recommend:
['انواع مولفه های هوش هیجانی | چطور']
['تکنیک های تبلیغات؛ ۱۳ تکنیکی که مخاطب را میخکوب می\u200cکند | چطور']
['معرفی مدل «کدهای هالند» و کاربردهای آن | چطور']
['ارزیاب

['خواص سیب؛ ۱۴ خاصیتی که سلامتی شما را بیمه می\u200cکند | چطور']
['خواص زالزالک برای سلامتی، نحوه مصرف و مضرات احتمالی آن | چطور']
['خواص خوراکی ها | چطور']
---user: abb71af7-7c147-75494-47c57-71e800151e295 ---
what user saw:
['خواص سیب؛ ۱۴ خاصیتی که سلامتی شما را بیمه می\u200cکند | چطور']
['مد و زیبایی | چطور']
['تونر صورت چیست و استفاده از آن چه فوایدی برای پوست دارد؟ | چطور']
what we recommend:
['رفع ترک های پوستی با روش\u200cهای پزشکی و چند درمان\u200c موثر خانگی | چطور']
['خواص لیمو شیرین؛ ۳۰ خاصیت درمانی یکی از غنی\u200cترین منابع ویتامین C | چطور']
['از بین بردن جای جوش؛ ۱۴ ماده طبیعی که صورت شما را پاکسازی می\u200cکند | چطور']
['۱۴ روش سریع برای درمان جوش صورت | چطور']
['انواع مدل های شلوار؛ ۱۷ مدل شلوار که هیچ\u200cوقت از مد نمی\u200cافتند | چطور']
['فواید ویتامین C برای بدن و عوارض کمبود آن | چطور']
['انواع دامن زنانه؛ ۱۸ مدل دامن زیبا به همراه نکاتی کاربردی برای خیاط\u200cها | چطور']
['روشن شدن پوست صورت با ۱۵ روش خانگی | چطور']
['انواع آستین؛ ۴۵ نوع آستین برای لباس\u200cهای

...

['بهترین فیلم های ۲۰۱۷ که در ذهن شما ماندگار خواهند شد | چطور']
['ترین های جهان | چطور']
---user: ddb67260-c7ff-4bf7-cb1c-628fc9e00526 ---
what user saw:
['بهترین فیلم های عاشقانه جهان که شما را در فضایی رمانتیک غرق می\u200cکنند | چطور']
['بهترین سریال های خارجی دنیا که بیننده را معتاد خود می\u200cکنند | چطور']
['بهترین سریال های جنایی دنیا که طرفداران این ژانر باید تماشا کنند | چطور']
what we recommend:
['بهترین فیلم های تاریخ سینما که حداقل یک بار باید ببینید | چطور']
['۲۵ فیلم برتر قرن ۲۱ تاكنون كه حتما باید ببینید | چطور']
['بهترین فیلم های اکشن؛ ۳۱ فیلم برتر برای دوستداران هیجان | چطور']
['بهترین فیلم های رازآلود جهان که ذهن شما را ساعت\u200cها درگیر خود می\u200cکنند | چطور']
['بهترین فیلم های تاریخی جهان که حقیقت را به شما نمایش می\u200cدهند | چطور']
['فیلم و سریال | چطور']
['۴۴ فیلم دیدنی از بهترین فیلم های جنگی تاریخ سینما | چطور']
['بهترین فیلم های 2018 که شما را پای تلویزیون میخکوب می\u200cکنند | چطور']
['۱۵ فیلم الهام\u200cبخش که تمام اعضای خانواده از آن لذت می\u200cبرند | چ

['بهترین فیلم های رازآلود جهان که ذهن شما را ساعت\u200cها درگیر خود می\u200cکنند | چطور']
['بهترین فیلم های 2018 که شما را پای تلویزیون میخکوب می\u200cکنند | چطور']
['۴۴ فیلم دیدنی از بهترین فیلم های جنگی تاریخ سینما | چطور']
['بهترین فیلم های کمدی؛ ۲۵ فیلم خنده\u200cدار دنیا با داستان\u200cهای هیجان\u200cانگیز | چطور']
['برای سفید شدن پوست صورت چه باید کرد؟ | چطور']
['از بین بردن جای جوش؛ ۱۴ ماده طبیعی که صورت شما را پاکسازی می\u200cکند | چطور']
---user: be1923ce-e6b09-930ca-aa2b2-20c3788c0728a ---
what user saw:
['درمان آفت دهان با ۲۰ روش آسان خانگی | چطور']
['انتخاب بهترین کرم پودر برای پوست جوش دار؛ معرفی ۱۳ کرم پودر مناسب | چطور']
['۱۴ روش سریع برای درمان جوش صورت | چطور']
what we recommend:
['از بین بردن جای جوش؛ ۱۴ ماده طبیعی که صورت شما را پاکسازی می\u200cکند | چطور']
['درمان جوش صورت با گلاب؛ ۱۱ روش فوق\u200cالعاده برای درمان جوش | چطور']
['درمان جوش سرسیاه با ۱۱ روش موفق و قطعی خانگی | چطور']
['مد و زیبایی | چطور']
['درمان جوش سرسفید با ۱۰ روشی که صورتتان را مثل آینه می\u200c

['چگونه باکلاس باشیم؛ ۲۶ نکته برای شیک برخورد کردن | چطور']
['چگونه خاص باشیم؛ نکاتی که شما را برجسته و متفاوت نشان می\u200cدهد | چطور']
['چگونه محبوب باشیم؛ ۲۱ توصیه\u200c برای جا باز کردن در دل دیگران | چطور']
what we recommend:
['کاریزما | چطور']
['چگونه بهترین باشیم؛ ۱۷ کاری که شما را از معمولی بودن نجات می\u200cدهد | چطور']
['چگونه دیگران را جذب کنیم؟ | چطور']
['چگونه زیبا شویم؛ نکاتی که باعث درخشش شما می\u200cشود | چطور']
['۸ راه علمی برای جذاب\u200cتر شدن | چطور']
['چگونه با شخصیت باشیم؟ | چطور']
['۱۰ کاری که شما را از چشم بقیه می\u200cاندازد | چطور']
['۹ رفتار جذاب و گیرا در افراد کاریزماتیک | چطور']
['چگونه شخصیت قوی داشته باشیم؟ | چطور']
['۱۷ عادت گفتاری افراد خیلی دوست داشتنی | چطور']
---user: 82f608fd-9ca7-4de5-cd24-49d11c01818d ---
what user saw:
['خواص کلم؛ ۱۷ خاصیت درمانی این سبزی سرشار از ویتامین K | چطور']
['خواص میگو و مضرات احتمالی آن | چطور']
['خواص ماهی سالمون؛ ۱۱ دلیل مصرف سالم\u200cترین خوراکی دنیا | چطور']
what we recommend:
['خواص خوراکی ها | چطور']
['خواص لیمو

[]
what we recommend:
['قرص کلردیازپوکساید یا لیبریوم؛ عوارض،\u200c موارد و نحوه مصرف آن | چطور']
['قرص ضد استرس؛ معرفی داروهای ضد اضطراب و روش\u200cهای جایگزین برای کاهش استرس | چطور']
['داروها | چطور']
['قرص پردنیزولون و هرآنچه که باید درباره آن بدانید | چطور']
['قرص مونته لوکاست برای چیست؟ نحوه مصرف، عوارض و تداخل دارویی آن | چطور']
['افسردگی چیست؛ ۹ نشانه و ۶ عامل این بیماری شایع | چطور']
['انواع قرص خواب؛ ۸ نوع قرص خواب آور و نحوهٔ مصرف آن\u200cها | چطور']
['۱۱ راه طبیعی برای افزایش ترشح سروتونین در مغز و کاهش افسردگی | چطور']
['عوارض استرس مزمن بر بدن چیست؟ | چطور']
['کاهش هورمون استرس (کورتیزول) با ۸ راه موثر | چطور']
---user: 4f1b2098-85bdb-ba961-1d23f-fc76d9b6d6af1 ---
what user saw:
['خواص جو دوسر؛ فوایدی که این خوراکی برای کاهش وزن و پوست شما دارد | چطور']
['راهنمای کامل رژیم دوکان، مراحل و برنامه غذایی آن | چطور']
['رژیم سبزیجات و تأثیر فوق العاده آن بر کاهش وزن | چطور']
what we recommend:
['رژیم و تغذیه | چطور']
['یک رژیم کم کربوهیدرات شامل چه خوراکی\u200cهایی می\u200cشود؟

['علت ایست قلبی و علائم آن؛ در برخورد با ایست قلبی چه باید کرد؟ | چطور']
['علائم بیماری قلبی در زنان؛ ۵ نشانه\u200cای که باید جدی بگیرید | چطور']
['ضربان قلب نرمال افراد در سنین مختلف چگونه است؟\u200c | چطور']
['بیماری ها | چطور']
['علت درد قفسه سینه و علائم آن چیست؟ | چطور']
['علت سکته قلبی در جوانان چیست؟ | چطور']
['۴۰ علت ضعف ناگهانی بدن | چطور']
['چطور خطر ابتلا به بیماری\u200cهای قلبی را کاهش دهیم؟ | چطور']
---user: f450a72d-7740-4c40-cc01-9d3ebe101ade ---
what user saw:
['بهترین فیلم های اکشن؛ ۳۱ فیلم برتر برای دوستداران هیجان | چطور']
[]
['پس انداز پول با ۴۴ روشی که برای روز مبادا باید بدانید | چطور']
['۲۰ سوال مهمی که برای توسعه فردی خودتان باید به آنها پاسخ دهید | چطور']
what we recommend:
['هزینه نگهداری سگ چقدر است؟ لیست کامل هزینه\u200cها همراه با جزئیات (مهر ۹۹) | چطور']
['بهترین فیلم های تاریخ سینما که حداقل یک بار باید ببینید | چطور']
['پول و ثروت | چطور']
['سبک زندگی | چطور']
['ترین های جهان | چطور']
['چگونه پولدار شویم؛ ۱۶ راه پولدار شدن که باید بدانید | چطور']
['چگونه